In [ ]:
# Imports 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xarray as xr
import folium
import numpy as np
import pandas as pd
import datetime as dt

from eodag import EODataAccessGateway
from eodag import setup_logging
from eodag.crunch import FilterProperty


from rasterio.crs import CRS
from rioxarray.merge import merge_arrays

import eotools.shortcut as eoshort
import eotools.loading as eoload

# Setup Verbose Values:
# 0: no logging and no progress bar
# 1: no logging but progress bars displayed
# 2: log at the INFO level
# 3: log at the DEBUG level (even more information)

setup_logging(verbose=1)

# EODAG - Search and Post Process

EODAG (Earth Observation Data Access Gateway) is a command line tool and a Python package for searching and downloading remotely sensed images while offering a unified API for data access regardless of the data provider.

EODAG gives you an easy way to access products from more than 10 providers, with more than 50 different product types (Sentinel 1, Sentinel 2, Sentinel 3, Landsat, etc.) that can be searched and downloaded.

## Step 1
### Configuration
In the configuration we pass the username and password from the Copernicus Dataspace Ecosystem (CDSE) to eodag. Also we define the path for the downloads.

In [ ]:
# Get Credentials from .env file and make dirs from paths.yml
# These functions are just shortcuts from a python script and just help to keep the notebooks short and simple
workspace = eoshort.read_paths(filepath='paths.yml')
dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace")

## Step 2
### Search
Searching for products is one of the most important features of eodag. Here you pass the desired parameters of your search.

In [ ]:
# 2. Search 
#Set Search Parameters and get catalog of products

latmin, latmax = 47.86, 48.407
lonmin, lonmax = 16.0, 16.9
extent = {'lonmin': lonmin, 'latmin': latmin, 'lonmax': lonmax, 'latmax': latmax}


search_results, total_count = dag.search(
    productType='S2_MSI_L2A',
    provider='cop_dataspace',
    #geom=extent,
    tileIdentifier="33UXP",
    start='2023-03-14',
    end='2023-03-21',
    cloudCover=100 #You can specify cloudcover here or later when filtering.
)

print(f"Got {len(search_results)} products and an estimated total number of {total_count} products.")

In [ ]:
# Folium Map
fmap = folium.Map(location=(np.array([latmin, latmax]).mean(), np.array([lonmin, lonmax]).mean()), zoom_start=9)
folium.Rectangle(bounds=[[latmin, lonmin],[latmax, lonmax]], color="red").add_to(fmap)
folium.GeoJson(
    data=search_results[:],  # SearchResult has a __geo_interface__ interface used by folium to get its GeoJSON representation, single results dont work (this [2:3] instead of [2])
    tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)
fmap

In [ ]:
#Plot Quicklooks of Search Results
eoshort.plot_quicklooks(search_results)

## Step 3
### Crunch
Crunching as defined in eodag is a way to filter the EO products contained in a SearchResult object. Several filters are available and further described in this document.

A SearchResult has a crunch() method that requires a filter instance as an argument, itself initialized with a dictionary that contains the required parameters. According to the filter used, some more kwargs may need to be passed to crunch(). The filters return a list of EOProducts.

The following Filter Options are available (Note `sr` is the search result):
- Filter by start and end date (`sr.filter_date()` or import `FilterDate` from eodag.crunch)
- Filter by geometry (`FilterOverlap`, )
- Filter by property (`FilterProperty`)
- Filter for online products (`sr.filter_online()` or `FilterProperty`)
- Filter the latest products intersecting a geometry (`sr.filter_latest_intersect()` or `FilterLatestIntersect`)




In [ ]:
# 3. Crunch (other Crunching Methods are described on EODAG Website)

#Filter for Cloudcover
filtered_products = search_results.crunch(
    FilterProperty(dict(cloudCover=80, operator="lt")))
print(f"{len(search_results) - len(filtered_products)} from {len(search_results)} products were filtered out by the property filter.")

#Filter already filterd products for online products
online_products = filtered_products.filter_online()
print(f"{len(filtered_products) - len(online_products)} from {len(online_products)} products are offline and have been filtered out.")

#plot_quicklooks(online_products)



## Step 4 
### Download

EOProducts can either be downloaded individually with ``download()`` or together with ``download_all()`` from a SearchResult. If the product has already been downloaded and the right directory has been configured, then it will not be downloaded again, as eodag checks for rexisting files.

In [ ]:
# Download multiple Products
products = online_products
paths = dag.download_all(products)


## Step 5
### Post Process

Simple Example of an NDVI Image.


In [ ]:
# Setting common Parameters for all further image processing
common_params = dict(
    crs=CRS.from_epsg(4326),               # the downloaded images are in 4326, don't reproject them
    resolution=0.0006,                     # but lower their resolution (0.0006 should be 60m in 100km)
    extent=(lonmin,latmin,lonmax,latmax)   # and zoom over/crop the area of interest
)

# Define basic Functions for future operations
def normalized_difference(a, b):
    return (a - b*1.)/(a + b)

In [ ]:
# Load Bands 4 and 8 into memory
RED = products[0].get_data(band="B04", **common_params)
NIR = products[0].get_data(band="B08", **common_params)

NDVI = normalized_difference(NIR, RED)

In [ ]:
# Plot the NDVI Image
NDVI.plot(cmap="RdYlGn", center=False, size=6, aspect='equal')